# National Health and Nutrition Examination Survey (NHANES)
[https://wwwn.cdc.gov/nchs/nhanes/](https://wwwn.cdc.gov/nchs/nhanes/)

Este laboratório usará dados extraídos do NHANES.

## NHANES datasets from 2013-2014

Foram extraídos dados do [NHANES 2013-2014](https://wwwn.cdc.gov/nchs/nhanes/continuousnhanes/default.aspx?BeginYear=2013).

Os dados estão disponíveis em arquivos CSV no diretório `/data/nhanes`. Lá está disponível a documentação do esquema dos arquivos.

# Tarefa 1 - Esquema Relacional

Analise os arquivos CSV que estão no diretório `/data/nhanes` e escreva um esquema relacional referente às tabelas, suas chaves primárias e estrangeiras (não é SQL ainda). Coloque a chave primária com dois underlines antes e dois depois:

\__teste\__

Pessoa( __id__, gender, age)
Med(__code__, name)
MedUse(__person_id__, __drug_code__ , days_use) 

## Criação das Tabelas em SQL e Importação de Dados

Considere o seguinte exemplo em que é criada uma tabela em SQL e importado dados do arquivo CSV `demographic-person.csv`.

In [1]:
%defaultDatasource jdbc:h2:mem:db

In [7]:
DROP TABLE IF EXISTS Pessoa;
CREATE TABLE Pessoa (
  Id VARCHAR(5) NOT NULL,
  Gender INTEGER,
  Age INTEGER,
  PRIMARY KEY(Id)
) AS SELECT
    id,
    gender,
    age
FROM CSVREAD('../../../data/nhanes/demographic-person.csv');

In [8]:
SELECT * FROM Pessoa;

# Tarefa 2 - Criação das Tabelas em SQL e Importação de Dados

Escreva instruções SQL para montar as duas tabelas restante, conforme definido no esquema relacional da **Tarefa 1**.

**Importante:** não defina chave primária para a tabela de uso de medicamento.

Durante a criação das tabelas, realize a importação dos dados do arquivo CSV nas tabelas criadas, conforme exemplo anterior.

In [9]:
DROP TABLE IF EXISTS Med;
DROP TABLE IF EXISTS MedUse;
CREATE TABLE Med (
  Code VARCHAR(6) NOT NULL,
  Name VARCHAR(200), 
  PRIMARY KEY(Code)
) AS SELECT
    Code,
    Name
FROM CSVREAD('../../../data/nhanes/medications-drug.csv');

CREATE TABLE MedUse (
  PersonId VARCHAR(5) NOT NULL,
  DrugCode VARCHAR(6) NOT NULL,
  DaysUse INTEGER,
  FOREIGN KEY(PersonId) 
      REFERENCES Pessoa(Id) 
        ON DELETE NO ACTION 
        ON UPDATE NO ACTION,
  FOREIGN KEY(DrugCode) 
    REFERENCES Med(Code) 
        ON DELETE NO ACTION 
        ON UPDATE NO ACTION,
) AS SELECT
    person_id,
    drug_code,
    days_use
FROM CSVREAD('../../../data/nhanes/medications-use.csv');

In [10]:
SELECT * FROM Med;

# Tarefa 3 - Consultas Simples

Execute as consultas a seguir.

a) Liste pessoas com idade acima de 60

In [11]:
Select * FROM Pessoa WHERE age>60

b) Liste as idades das pessoas cadastradas (sem repetições)

In [12]:
SELECT DISTINCT age FROM Pessoa 

c) Liste código das pessoas, idade, código dos medicamentos que usaram e dias de uso

In [13]:
SELECT Pessoa.Id, Pessoa.Age, MedUse.DrugCode, MedUse.DaysUse FROM Pessoa, MedUse Where Pessoa.Id = MedUse.PersonId 

d) Liste código das Pessoas, idade, nome dos medicamentos que usaram e quantos dias de uso

In [14]:
SELECT Pessoa.Id, Pessoa.Age, Med.Name, MedUse.DaysUse FROM Pessoa, Med, MedUse Where Pessoa.Id = MedUse.PersonId and Med.Code= MedUse.DrugCode

e) Liste nome dos medicamentos que foram usados por pessoas (sem repetições)

In [17]:
SELECT DISTINCT Med.Name FROM Pessoa, Med, MedUse Where Pessoa.Id = MedUse.PersonId and Med.Code= MedUse.DrugCode

f) Liste nome dos medicamentos usados por mais de 2.000 dias (sem repetições)

In [45]:
SELECT Med.Name, SUM(MedUse.DaysUse) Dias_Usados 
        FROM Med, MedUse 
        Where Med.Code=MedUse.DrugCode 
        GROUP BY(Med.Name) 
        HAVING Dias_Usados>2000

In [44]:
SELECT Medic.Name FROM (
    SELECT Med.Name, SUM(MedUse.DaysUse) Dias_Usados 
        FROM Med, MedUse 
        Where Med.Code=MedUse.DrugCode 
        GROUP BY(Med.Name) 
        HAVING Dias_Usados>2000) Medic